In [1]:
import Engine
import chess
import numpy as np
import time

# The Search Algorithm

Here I will create the algorithm that decides which variations to play. I will be using a min max algorithm. This is basically an algorithm that skips variations that can't affect the algorithm.

Lets assume we are playing as white. The idea is that in each step (turn for a player) the algorithm will either minimize the eval or maximize it. Variations can be seen as trees (with corresponding roots, nodes and leafs). Suppose we evaluate to depth = n, where it is blacks turn. The algorithm will start by evaluating all the leafs of one of the previous nodes, since it is black's turn we assume black will try to play the best move (minimize evaluation). Therefore from that set of leafs, the only relevant leaf is the one with minimum evaluation. However we will chose the node that has min(set of leafs for node) the maximum from all nodes. This means that when evaluating the leafs of the next node we can stop (and skip to the next set of leafs for another node) whenever we find a leaf whose evaluation is less than the minimum at that point between all the leafs evaluated.

We can do this for each level, starting from the leafs of the tree upward toward the root.


THINGS To do:
* Make engine think while it ins't its move (using the i's in alpha_beta algorithm) - **Done on algorithm, need to finish on engine.**
* Use Pypy interpreter instead of CPython, since it is several magnitudes faster (except for numpy, so maybe we should avoid numpy).


In [2]:
# Test mod 8 methods
start_time = time.time()
for i in range(64):
    i % 8
end_time = time.time()
print(start_time-end_time)

start_time = time.time()
for i in range(64):
    divmod(i,8)
end_time = time.time()
print(start_time-end_time)

# Test mod 10 methods
start_time = time.time()
for i in range(64):
    i%10
end_time = time.time()
print(start_time-end_time)

start_time = time.time()
for i in range(64):
    divmod(i,10)
end_time = time.time()
print(start_time-end_time)

start_time = time.time()
for i in range(64):
    int(str(i)[-1])
end_time = time.time()
print(start_time-end_time)

-5.984306335449219e-05
-5.602836608886719e-05
-4.38690185546875e-05
-4.887580871582031e-05
-7.104873657226562e-05


The fastest is i%10 however i%8 is also pretty fast, also it depends on the interpreter so it would be reasonable to try this with Pypy too. I will choose i%8, therefore my position will be a list of 64 elements. **Taking 100 has one further advantage, we are creating a 10 by 10 board and when checking if pieces are at the end of the board this is useful**. I will be dividing the board in 4 squares and precomputing the moves on one of the 4 by 4 squares say (a1, a4, d1,d4). And using the symmetries I will be able to precompute all possible moves.

In [3]:
board = [ 'R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R',          
         'P', 'P', 'r', 'P', 'klk', 'P', 'P', 'P',          
         '0', '0', '0', '0', '0', '0', '0', '0',          
         '0', '0', 'k', '0', '0', '0', '0', '0',       
         '0', '0', '0', '0', '0', '0', '0', '0',          
         '0', '2', 'm', '0', '0', '0', '0', '0',          
         'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p',          
         'r', 'n', 'b', 'q', 'k', 'b', 'n', 'k' ]

# Quadrant h8
flipboard = list(reversed(board)) # Flip the board pieces horizontally
flipsquarelist = list(reversed(range(64)))# correct
flipsquare = flipsquarelist[3]
print('Quadrant_h8:', flipboard) # Este esta hecho para h8


# Quadrant a8
flipboard = [square for row in [board[i:i+8] for i in range(0, 64, 8)][::-1] for square in row]
flipsquarelist = [square for row in [list(range(64))[::-1][i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
flipsquare = flipsquarelist[3]
print('Quadrant_a8:', flipboard)

# Quadrant h1
flipboard = [square for row in [board[i:i+8][::-1] for i in range(0, 64, 8)] for square in row] 
flipsquarelist = [square for row in [list(range(64))[i:i+8][::-1] for i in range(0, 64, 8)] for square in row]
flipsquare = flipsquarelist[3]
print('Quadrant_h1:', flipboard) # Este esta hecho para h1

Quadrant_h8: ['k', 'n', 'b', 'k', 'q', 'b', 'n', 'r', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', '0', '0', '0', '0', '0', 'm', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'k', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'P', 'P', 'P', 'klk', 'P', 'r', 'P', 'P', 'R', 'N', 'B', 'K', 'Q', 'B', 'N', 'R']
Quadrant_a8: ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'k', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', '0', '2', 'm', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'k', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'P', 'P', 'r', 'P', 'klk', 'P', 'P', 'P', 'R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
Quadrant_h1: ['R', 'N', 'B', 'K', 'Q', 'B', 'N', 'R', 'P', 'P', 'P', 'klk', 'P', 'r', 'P', 'P', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'k', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'm', '2', '0', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'k', 'n', 'b', 'k', 'q', 'b', 'n

In [4]:
def pretty_print(board):
    # Create a list of rank labels (1 to 8) and file labels (a to h)
    ranks = [str(i) for i in range(8, 0, -1)]
    files = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']


    # Print each rank of the board
    for i in range(8):
        rank = ''.join(str(board[(8-i-1) * 8 + j]) for j in range(8))
        rank = rank.replace('0', '.')
        rank = rank.replace('K', '♔')
        rank = rank.replace('Q', '♕')
        rank = rank.replace('R', '♖')
        rank = rank.replace('B', '♗')
        rank = rank.replace('N', '♘')
        rank = rank.replace('P', '♙')
        rank = rank.replace('k', '♚')
        rank = rank.replace('q', '♛')
        rank = rank.replace('r', '♜')
        rank = rank.replace('b', '♝')
        rank = rank.replace('n', '♞')
        rank = rank.replace('p', '♟')
        print(rank )

    

In [5]:
def fen_to_position(fen):
    board = ['0']*64
    fen_parts = fen.split()
    fen_board = fen_parts[0].split('/')
    for i in range(len(fen_board)):
        fen_row = fen_board[i]
        j = 0
        for c in fen_row:
            if c.isdigit():
                j += int(c)
            else:
                board[(7-i)*8+j] = c
                j += 1
    
    en_passant = chess.SQUARE_NAMES.index(fen_parts[3]) if fen_parts[3] != '-' else -1
    
    white_castling_rights = [False]*2
    if 'K' in fen_parts[2]:
        white_castling_rights[0] = True
    if 'Q' in fen_parts[2]:
        white_castling_rights[1] = True
    black_castling_rights = [False]*2
    if 'k' in fen_parts[2]:
        black_castling_rights[0] = True
    if 'q' in fen_parts[2]:
        black_castling_rights[1] = True
    
    turn = fen_parts[1] == 'w'
    
    return Engine.Position(board, en_passant, white_castling_rights, black_castling_rights, [], turn)


In [6]:
def position_to_fen(position):
    board = chess.Board()
    board.set_board_fen(position.board_fen())
    fen_parts = [
        board.board_fen(),
        'w' if position.turn else 'b',
        ''.join(['K' if position.castling_rights[0] else '',
                 'Q' if position.castling_rights[1] else '',
                 'k' if position.castling_rights[2] else '',
                 'q' if position.castling_rights[3] else '']),
        chess.SQUARE_NAMES[position.en_passant] if position.en_passant != -1 else '-',
        str(position.halfmove_clock),
        str(position.fullmove_number)
    ]
    return ' '.join(fen_parts)

### The Simple Algorithms

I am creating two simple algorithms, one which runs with the python chess library (alpha_beta_from_fen) and the other with my move generator. My move generator appears to make the algorithm 2 times faster.

In [7]:
def alpha_beta_from_fen(fen, depth, alpha, beta, maximize, evaluate_position): 
    # maximize is a boolean variable (True if we want to maximize the evaluation (we are white))
    # alpha is the current best evaluation for white, it will start at -infinity
    # beta is the current best evaluation for black, it will start at +infinity
    
    board = chess.Board(fen = fen)
    legal_moves = list(board.legal_moves)
    
    if depth == 0:
        return evaluate_position(fen), []
    
    best_move = None
    
    if maximize:
        value = -1000
        for move in legal_moves:
            board.push(move)
            child_value = alpha_beta_from_fen(board.fen(), depth - 1, alpha, beta, False, evaluate_position)[0]
            if child_value > value:
                value = child_value
                best_move = move
            board.pop() # Unmake the last move
            alpha = max(alpha, value)
            if alpha >= beta:
                break
    else:
        value = 1000
        for move in legal_moves:
            board.push(move)
            child_value = alpha_beta_from_fen(board.fen(), depth - 1, alpha, beta, True, evaluate_position)[0]
            if child_value < value:
                value = child_value
                best_move = move
            board.pop() 
            beta = min(beta, value)
            if beta <= alpha:
                break
    
    return value, best_move

In [8]:
def engine_fen(fen, is_maximizing_player, is_our_move, depth): # ¡¡¡ FIX AND ADD VARIATIONS!!!
    # is_maximizing_player is set to True if we are white
    # alpha is the current best evaluation for white, it will start at -infinity
    # beta is the current best evaluation for black, it will start at +infinity
    start_time = time.time()
    best_move = None
    board = chess.Board(fen)
    legal_moves = list(board.legal_moves)

    if is_our_move:
        if is_maximizing_player:
            alpha = -np.inf
            beta = np.inf
            best_value = -np.inf
        else:
            alpha = -np.inf
            beta = np.inf
            best_value = np.inf
        
        for move in legal_moves:
            board.push(move)
            new_fen = board.fen()
            value = alpha_beta_from_fen(new_fen, depth - 1, alpha, beta, not is_maximizing_player, Engine.evaluate_position_from_fen)[0]
            board.pop()

            if is_maximizing_player and value > best_value:
                best_move = move
                best_value = value
            elif is_maximizing_player==False and value < best_value:
                best_move = move
                best_value = value
        end_time = time.time()
        elapsed_time = end_time - start_time
        return ["Time taken:", elapsed_time, "seconds", "Best move: ", best_move, "Evaluation: ", best_value]
    else: # Now we make the engine think while it isn't it's turn
        Dict = {}
        for move in legal_moves:
            board.push(move)
            new_fen = board.fen()
            best_move = None
            Dict[move]=(engine(fen, is_maximizing_player, is_our_move, depth+1)[4], 
                        engine(fen, is_maximizing_player, is_our_move, depth+1)[6])
        return(Dict) # This is a dictionary of the form {move: best response, evaluation}
        

In [9]:
engine_fen('r2qkbnr/1bp2ppp/1pnp4/pB2p3/3PP3/2N2N2/PPP2PPP/R1BQ1RK1 w kq - 2 7', True, True, 5)

['Time taken:',
 177.2241358757019,
 'seconds',
 'Best move: ',
 Move.from_uci('d4e5'),
 'Evaluation: ',
 400]

This is the best move, in a position where we are black and it is our move. For a search of depth 5, it takes 154 seconds which should be improved, it is finding the best move in a simple capture. Note that we are using a very simple evaluation function which will be faster than the true one.

In [10]:
def alpha_beta_from_position(position, depth, alpha, beta, maximize):
    # maximize is a boolean variable (True if we want to maximize the evaluation (we are moving))
    # alpha is the current best evaluation for us, it will start at -infinity
    # beta is the current best evaluation for opponent, it will start at +infinity
    if depth == 0 and maximize:
        # If in the last move it is our turn, the position will be evaluated as if we are white, since we are 
        # the maximizing player, this is the correct sign
        return Engine.evaluate_position_from_board(position.board), 0
    if depth == 0 and not maximize:
        # If in the last move is the opponents turn, the position will be evaluated as if we are black, since
        # we are the maximizing player, we change the sign
        return -Engine.evaluate_position_from_board(position.board), 0 
    
    legal_moves = position.gen_moves()
    best_move = None
    
    if maximize: # If we have to move, we want to maximize. This is ensured by the above
        value = -10000 # This is the best evaluation for white in the child_values
        for move in legal_moves:
            position1 = position
            child_value = alpha_beta_from_position(position1.move(move), depth - 1, alpha, beta, False)[0]
            if child_value > value:
                value = child_value
                best_move = move
            alpha = max(alpha, value)
            if alpha >= beta:
                break # Because if our best move is better than the best in another set of moves that lead from
                      # a different move from opponent, then the opponent will choose the other move.
                
    else: # If opponent has to move
        value = 10000
        for move in legal_moves:
            position1 = position
            child_value = alpha_beta_from_position(position1.move(move), depth - 1, alpha, beta, True)[0]
            if child_value < value:
                value = child_value
                best_move = move
            beta = min(beta, value)
            if alpha >= beta: # Because if our opponents best move is better (less than) than the best 
                break         # in another set of moves that lead from a different move from us, 
                              # then we will choose the other move.
    
    return value, best_move 


In [11]:
import numpy as np
def engine_position(position, is_our_move, depth): 
    # alpha is the current best evaluation for white, it will start at -1000
    # beta is the current best evaluation for black, it will start at +1000
    start_time = time.time()
    best_move = None
    legal_moves = position.gen_moves()

    if is_our_move:
        alpha = -10000 # Current best evaluation for us
        beta = 10000 # Current best evaluation for opponent
        best_value = -10000
        best_value, best_move = alpha_beta_from_position(position, depth, alpha, beta, True)
        end_time = time.time()
        elapsed_time = end_time - start_time
        return ["Time taken:", elapsed_time, "seconds", "Best move: ", best_move, "Evaluation: ", best_value]
    else: # Now we make the engine think while it isn't it's turn
        Dict = {}
        for move in legal_moves:
            new_position = position.move(move)
            best_move = None
            Dict[move]=(engine_position(new_position, is_maximizing_player, is_our_move, depth+1)[4], 
                        engine_position(new_position, is_maximizing_player, is_our_move, depth+1)[6])
        return(Dict) # This is a dictionary of the form {move: best response, evaluation}


In [12]:
engine_position(fen_to_position('r2qkbnr/1bp2ppp/1pnp4/pB2p3/3PP3/2N2N2/PPP2PPP/R1BQ1RK1 w kq - 2 7'), True, 5)


['Time taken:',
 0.05463695526123047,
 'seconds',
 'Best move: ',
 Move(i=2, j=11, prom='', score=0),
 'Evaluation: ',
 10000]

In [13]:
engine_position(fen_to_position('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'), True, 6)

['Time taken:',
 0.0698859691619873,
 'seconds',
 'Best move: ',
 Move(i=1, j=11, prom='', score=0),
 'Evaluation: ',
 10000]

### Improving the Algorithms 

There are several ways to improve the alpha-beta algorithm for chess:

* Iterative Deepening: This technique involves running the alpha-beta search multiple times with increasing depth limits. It allows the program to quickly find a good move at lower depths, and then continue searching deeper if there is time remaining.

* Killer moves: This are moves stored during the search when iterative deepening. If a move creates a cutoff during one step of iterative deepening (running alphabeta on a fixed depth), then it is stored and on the next step of iterative deepening it will be tried the first move.

* Transposition Tables: This technique involves storing previously computed evaluations in a hash table, so that the algorithm can avoid redundant evaluations. This can be particularly useful when searching the same position from different move orders.

* Move Ordering: This technique involves ordering the moves so that the most promising moves are searched first. This can lead to pruning more branches and searching deeper in the remaining branches. **DONE WITH CAPTURES AND CEHCKS**

* Quiescence Search: This technique involves extending the search depth for positions with captures and checks, as these are often critical in chess. This can help avoid the "horizon effect" where a position looks good at the current depth, but leads to a bad position in the next ply. **DONE**

* Null Move Pruning: This technique involves temporarily passing the turn to the opponent and then evaluating the position. If the evaluation is significantly worse than the current best score, then the move leading to that position is unlikely to be good, and the algorithm can skip searching that move.

* Futility Pruning: This technique involves pruning branches where the evaluation is already significantly worse than the current best score. This can reduce the search depth and speed up the algorithm.

In [14]:
Engine.Engine.Tell_Move(Engine.Move(1,8,'Q',0))

'b1a2Q'

### Checkmates (The engine makes them all)

In [15]:
Engine.Engine.Search(fen_to_position('8/8/8/8/8/2R5/1R6/6k1 w - - 0 1'), True, 2)


['Time taken:',
 0.051016807556152344,
 'seconds',
 'Best move: ',
 Move(i=18, j=2, prom='', score=3),
 'Evaluation: ',
 1003]

In [16]:
print(Engine.Engine.Search(fen_to_position('6k1/4Rppp/8/8/8/8/5PPP/6K1 w - - 0 1'), True, 2))


['Time taken:', 0.06203961372375488, 'seconds', 'Best move: ', Move(i=52, j=60, prom='', score=3), 'Evaluation: ', 1003]


In [17]:
print(Engine.Engine.Search(fen_to_position('2r1r1k1/5ppp/8/8/Q7/8/5PPP/4R1K1 w - - 0 1'), True, 4))


['Time taken:', 0.7406039237976074, 'seconds', 'Best move: ', Move(i=4, j=60, prom='', score=5), 'Evaluation: ', 1001]


In [18]:
print(Engine.Engine.Search(fen_to_position('6k1/3qb1pp/4p3/ppp1P3/8/2PP1Q2/PP4PP/5RK1 w - - 0 1'), True, 4))


['Time taken:', 2.3451731204986572, 'seconds', 'Best move: ', Move(i=21, j=53, prom='', score=3), 'Evaluation: ', 1001]


In [19]:
print(Engine.Engine.Search(fen_to_position('R7/4kp2/5N2/4P3/8/8/8/6K1 w - - 0 1'), True, 4))


['Time taken:', 2.2818009853363037, 'seconds', 'Best move: ', Move(i=56, j=60, prom='', score=3), 'Evaluation: ', 1003]


In [20]:
print(Engine.Engine.Search(fen_to_position('5r1b/2R1R3/P4r2/2p2Nkp/2b3pN/6P1/4PP2/6K1 w - - 0 1'), True, 4))


['Time taken:', 50.45259380340576, 'seconds', 'Best move: ', Move(i=52, j=54, prom='', score=3), 'Evaluation: ', 1001]


### Some simple tactics (The engine makes the correct move, except for one)

In [21]:
pretty_print(fen_to_position('r2qkbnr/1bp2ppp/1pnp4/pB2p3/3PP3/2N2N2/PPP2PPP/R1BQ1RK1 w kq - 2 7').board)
print(Engine.Engine.Search(fen_to_position('r2qkbnr/1bp2ppp/1pnp4/pB2p3/3PP3/2N2N2/PPP2PPP/R1BQ1RK1 w kq - 2 7'), True, 3), 'Best move should be: 27, 35')


♜..♛♚♝♞♜
.♝♟..♟♟♟
.♟♞♟....
♟♗..♟...
...♙♙...
..♘..♘..
♙♙♙..♙♙♙
♖.♗♕.♖♔.
['Time taken:', 25.169307947158813, 'seconds', 'Best move: ', Move(i=33, j=42, prom='', score=5), 'Evaluation: ', 600] Best move should be: 27, 35


In [22]:
pretty_print(fen_to_position('1kr2b1r/ppp2ppp/2n2n2/3p4/3P2qP/2N1P1P1/PPP2P2/R1BKQ2R w KQk - 0 1').board)
print(list(fen_to_position('1kr2b1r/ppp2ppp/2n2n2/3p4/3P2qP/2N1P1P1/PPP2P2/R1BKQ2R w KQk - 0 1').out_of_attack_moves()))
print(list(fen_to_position('1kr2b1r/ppp2ppp/2n2n2/3p4/3P2qP/2N1P1P1/PPP2P2/R1BKQ2R w KQk - 0 1').ordered_moves()))


.♚♜..♝.♜
♟♟♟..♟♟♟
..♞..♞..
...♟....
...♙..♛♙
..♘.♙.♙.
♙♙♙..♙..
♖.♗♔♕..♖
[Move(i=3, j=11, prom='', score=0), Move(i=4, j=12, prom='', score=0), Move(i=13, j=21, prom='', score=0), Move(i=18, j=12, prom='', score=0)]
[Move(i=3, j=11, prom='', score=0), Move(i=4, j=12, prom='', score=0), Move(i=13, j=21, prom='', score=0), Move(i=18, j=12, prom='', score=0)]


In [23]:
pretty_print(fen_to_position('1k6/ppp3q1/8/4r3/8/8/3B1PPP/R4QK1 w - - 0 1').board)
print(Engine.Engine.Search(fen_to_position('1k6/ppp3q1/8/4r3/8/8/3B1PPP/R4QK1 w - - 0 1'), True, 4), 'Best move should be: 11, 18')


.♚......
♟♟♟...♛.
........
....♜...
........
........
...♗.♙♙♙
♖....♕♔.
['Time taken:', 131.82509589195251, 'seconds', 'Best move: ', Move(i=11, j=18, prom='', score=0), 'Evaluation: ', 400] Best move should be: 11, 18


In [24]:
pretty_print(fen_to_position('4k3/6p1/5p1p/4n3/8/7P/5PP1/4R1K1 w - - 0 1').board)
print(Engine.Engine.Search(fen_to_position('4k3/6p1/5p1p/4n3/8/7P/5PP1/4R1K1 w - - 0 1'), True, 4), 'Best move should be: 13, 29')


....♚...
......♟.
.....♟.♟
....♞...
........
.......♙
.....♙♙.
....♖.♔.
['Time taken:', 2.6971991062164307, 'seconds', 'Best move: ', Move(i=13, j=29, prom='', score=0), 'Evaluation: ', 400] Best move should be: 13, 29


In [25]:
pretty_print(fen_to_position('r4rk1/pp1p1ppp/1qp2n2/8/4P3/1P1P2Q1/PBP2PPP/R4RK1 w - - 0 1').board)
print(Engine.Engine.Search(fen_to_position('r4rk1/pp1p1ppp/1qp2n2/8/4P3/1P1P2Q1/PBP2PPP/R4RK1 w - - 0 1'), True, 4), 'Best move should be: 9, 45')


♜....♜♚.
♟♟.♟.♟♟♟
.♛♟..♞..
........
....♙...
.♙.♙..♕.
♙♗♙..♙♙♙
♖....♖♔.
['Time taken:', 56.54353308677673, 'seconds', 'Best move: ', Move(i=9, j=45, prom='', score=2), 'Evaluation: ', 400] Best move should be: 9, 45


In [26]:
pretty_print(fen_to_position('2k2bnr/pp2pp1p/6p1/5n2/8/7B/1PP1P1PP/2B1K1NR w Kk - 0 1').board)
print(Engine.Engine.Search(fen_to_position('2k2bnr/pp2pp1p/6p1/5n2/8/7B/1PP1P1PP/2B1K1NR w Kk - 0 1'), True, 2), 'Best move should be: 12, 28')


..♚..♝♞♜
♟♟..♟♟.♟
......♟.
.....♞..
........
.......♗
.♙♙.♙.♙♙
..♗.♔.♘♖
['Time taken:', 0.39755725860595703, 'seconds', 'Best move: ', Move(i=12, j=28, prom='', score=0), 'Evaluation: ', 200] Best move should be: 12, 28


### A game


Now I'm going to create a function that using the position encoder, also gets the history matrices and repetition matrices.

In [27]:
def max_string_count(str_list):
    counts = {}
    for s in str_list:
        if s in counts:
            counts[s] += 1
        else:
            counts[s] = 1
    return max(counts.values())
# I HAVE CHECKED THAT THIS ONE IS RUNNING CORRECTLY

In [28]:
def game_encoder(pgn):
    hist_1 = [] #most recent position
    hist_2 = []
    hist_3 = []
    hist_4 = []
    hist_5 = []
    hist_6 = [] #6th most recent position
    for i in range(len(encode_position('rnbqkbnr/pppppppp/8/8/8/1P6/P1PPPPPP/RNBQKBNR b KQkq - 0 1'))):
        hist_1.append(np.zeros((8,8)))
        hist_2.append(np.zeros((8,8)))
        hist_3.append(np.zeros((8,8)))
        hist_4.append(np.zeros((8,8)))
        hist_5.append(np.zeros((8,8)))
        hist_6.append(np.zeros((8,8)))
    histories = [hist_1, hist_2, hist_3, hist_4, hist_5, hist_6]
    Fens = pgn_to_fenslist(pgn)
    for fen in Fens:
        histories[5] = histories[4]
        histories[4] = histories[3]
        histories[3] = histories[2]
        histories[2] = histories[1]
        histories[1] = histories[0]
        histories[0] = encode_position(fen)
    matrices_position = encode_position(Fens[-1])
    r = max_string_count(Fens)-1
    if r == 0:
        R_1 = np.zeros((8,8))
        R_2 = np.zeros((8,8))
    if r == 1:
        R_1 = np.ones((8,8))
        R_2 = np.zeros((8,8))
    else:
        R_1 = np.zeros((8,8))
        R_2 = np.ones((8,8))
    klk = [matrices_position, R_1, R_2, hist_1, hist_2, hist_3, hist_4, hist_5, hist_6]
    flat_list = [item for sublist in klk for item in sublist]
    return flat_list